_A Python code that records audio via microphone, applies wavelet-based noise reduction, saves the cleaned audio to a WAV file, and then plays it back on macOS._
-------------------------
**Explanation:**

1. The code imports necessary modules for audio capture, processing, and file handling.
2. It uses PyAudio to record 5 seconds of audio from the microphone.
3. Then, it applies Discrete Wavelet Transform (DWT) for basic denoising using thresholding.
4. The cleaned audio is saved as a `.wav` file.
5. Finally, it uses macOS’s `afplay` command to play the audio automatically.


In [ ]:
import pyaudio
import numpy as np
import pywt
import wave
import os

# Constants
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK_SIZE = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "cleaned_audio.wav"

# Set up PyAudio object
p = pyaudio.PyAudio()

# Open microphone input stream
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK_SIZE)

print("Recording...")
frames = []
for _ in range(0, int(RATE / CHUNK_SIZE * RECORD_SECONDS)):
    data = stream.read(CHUNK_SIZE)
    frames.append(data)
print("Finished recording")

# Convert to numpy array
audio_data = np.frombuffer(b''.join(frames), dtype=np.int16)

# Wavelet denoising
wavelet_name = 'db4'
level = 3
threshold = 0.04 * np.max(np.abs(audio_data))
coeffs = pywt.wavedec(audio_data, wavelet_name, level=level)
new_coeffs = [coeffs[0]] + [pywt.threshold(c, threshold, mode='soft') for c in coeffs[1:]]
new_audio = pywt.waverec(new_coeffs, wavelet_name)

# Ensure length consistency
new_audio = np.pad(new_audio, (0, len(audio_data) - len(new_audio)), mode='constant')
new_audio = new_audio.astype(np.int16)

# Save cleaned audio to WAV
with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(new_audio.tobytes())

# Clean up
stream.stop_stream()
stream.close()
p.terminate()

print("Saved cleaned audio to", WAVE_OUTPUT_FILENAME)

# Play the audio (macOS only)
print("Playing cleaned audio...")
os.system(f"afplay {WAVE_OUTPUT_FILENAME}")